#Buiding dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
!pip install PyPDF2
import PyPDF2
from lxml import etree
from tqdm.auto import tqdm
!pip install torch
!pip install -Uq transformers
!pip install -r '/content/drive/MyDrive/Datasests/Scisummnet/requirements.txt'

     |████████████████████████████████| 77 kB 3.3 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=7fbe9cd731e0c0a7510a81a154767849e8b516e4773abf87e4b214860e4c3231
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 37.7 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 636 kB 35.4 MB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 264 kB 11.1 MB/s 
     |████████████████████████████████| 1.2 MB 39.6 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 118 kB 45.8 MB/s 
     |████████████████████████████████| 243 kB 45.0 MB/s 
     |████████████████████████████████| 2.2 MB 32.9 MB/s 
     |█████████████████████

In [ ]:
root_path = '/content/drive/MyDrive/Datasests/Scisummnet/scisummnet_release1.1__20190413 (3).zip (Unzipped Files)/scisummnet_release1.1__20190413/top1000_complete'

In [ ]:
# getting list of all files and directories in the specified directory
pathlist = os.listdir(root_path)

In [ ]:
parser = etree.XMLParser(remove_blank_text=True)
pap_text = []
summaries = []
for folder in tqdm(pathlist):
    tree = etree.parse(root_path+'/'+folder+'/Documents_xml/'+folder+'.xml', parser)
    paper = []
    for s in tree.iter():
        if s.text is not None: paper.append(s.text)

    with open(root_path+'/'+folder+'/summary/'+folder+'.gold.txt') as f:
        lines = f.read()

    pap_text.append(' '.join(paper))
    summaries.append(lines)


  0%|          | 0/1009 [00:00<?, ?it/s]

In [ ]:
scisum_df = pd.DataFrame({'text': pap_text, 'summary': summaries})
scisum_df.to_csv('./scisumm.csv', index=False)

#Splitting the dataset

In [ ]:
scisum_df = pd.read_csv('/content/drive/MyDrive/Datasests/Scisummnet/scisumm.csv')

# Manual train-test split
np.random.seed(10)
msk = np.random.rand(len(scisum_df)) < 0.8
scisum_train_df = scisum_df[msk]
scisum_test_df = scisum_df[~msk]
scisum_train_df.to_csv('/content/drive/MyDrive/Datasests/Scisummnet/scisumm_train.csv')
scisum_test_df.to_csv('/content/drive/MyDrive/Datasests/Scisummnet/scisumm_test.csv')

In [ ]:
!git clone https://github.com/huggingface/transformers.git
os.chdir("/content/transformers")
!pwd
!pip install -e .

!python '/content/drive/MyDrive/Datasests/Scisummnet/run_summarization.py' \
    --model_name_or_path facebook/bart-large-cnn \
    --do_train \
    --do_eval \
    --train_file '/content/drive/MyDrive/Datasests/Scisummnet/scisumm_train.csv' \
    --validation_file '/content/drive/MyDrive/Datasests/Scisummnet/scisumm_test.csv' \
    --text_column text \
    --summary_column summary \
    --source_prefix "summarize: " \
    --output_dir "/content/drive/MyDrive/Datasests/Scisummnet/Summarization Output/" \
    --overwrite_output_dir \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --predict_with_generate

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'transformers': No such file or directory
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.2
    Uninstalling transformers-4.9.2:
      Successfully uninstalled transformers-4.9.2
  Running setup.py develop for transformers
08/27/2021 12:10:40 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/27/2021 12:10:40 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

PATH = "/content/drive/MyDrive/Datasests/Scisummnet/Summarization Output/checkpoint-2000"
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained(PATH, local_files_only=True)
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(PATH, local_files_only=True)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Datasests/Scisummnet/scisumm_test.csv')
gen_summs = []
for paper in tqdm(test_df.text.values[:203]):
#for paper in tqdm(scis_df.text.values[:5]):

  ARTICLE_TO_SUMMARIZE = paper
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt', truncation=True)

  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=500, early_stopping=True)
  gen_summs.append([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

gen_df = pd.DataFrame({'generated_summary':gen_summs})
gen_df.head()

gen_df.generated_summary = gen_df.generated_summary.apply(lambda x: x[0])
gen_df.head()

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge2', 'rouge3'])
scores = []
#mod: sharmaa4 for i in range(500):
for i in range(203):
  
  scores.append(scorer.score(test_df.iloc[i].summary, gen_df.iloc[i].generated_summary))


r2r = []
r2f = []
r3f = []
#for i in range(250):
for i in range(203):
  r2r.append(scores[i]['rouge2'].recall)
  r2f.append(scores[i]['rouge2'].fmeasure)
  r3f.append(scores[i]['rouge3'].fmeasure)


print('rouge2 - recall: '+ str(np.mean(r2r)))
print('rouge2 - fmeasure: '+ str(np.mean(r2f)))
print('rouge3 - fmeasure: '+ str(np.mean(r3f)))

gen_df.to_csv('./nonpt_first500.csv', index=False)

  0%|          | 0/203 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


rouge2 - recall: 0.531533252893803
rouge2 - fmeasure: 0.6378251993752988
rouge3 - fmeasure: 0.6190063056925416


pretrained_positional_embedding_sizes = 1024 for every model

In [ ]:
def gen_pdf_summary(pdf_file_path):
    pdf_obj = PyPDF2.PdfFileReader(pdf_file_path)
    page_no = pdf_obj.getNumPages()
    text = ""
    for i in range(1, page_no):
        text += pdf_obj.getPage(i).extractText()
    
    inputs_1 = tokenizer(text[0:1024], max_length=1024, return_tensors='pt', truncation=True)
    inputs_2 = tokenizer(text[1025:2048], max_length=1024, return_tensors='pt', truncation=True)
    inputs_3 = tokenizer(text[2048:3072], max_length=1024, return_tensors='pt', truncation=True)
    inputs_4 = tokenizer(text[3072:], max_length=1024, return_tensors='pt', truncation=True)

    # Generate Summary
    summary_ids_1 = model.generate(inputs_1['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_1 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_1]

    summary_ids_2 = model.generate(inputs_2['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_2 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_2]

    summary_ids_3 = model.generate(inputs_3['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_3 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_3]

    summary_ids_4 = model.generate(inputs_4['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_4 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_4]
  
    summary = summary_1[0]+summary_2[0]+summary_3[0]+summary_4[0]
    return summary


In [ ]:
gen_pdf_summary("/content/1909.01716.pdf")

To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.) [_tensor.py:575]


'Asolidsummaryofthe\ncontent\nFrom theauthors\' point of\nview, it may fail to understand the actual\nimpact\nofthe paper on\ntheresearch community.\nAdditionally, the authors of a\npaper may have difficulty understanding the actual pragmatic impact\nof the paper on the literature community, and may not be able to tackle the various problems in\npapers summarization.\nOurcorpus, which contain the information network of ACLAnthIn this paper we present two approaches for estimating the impact of unsupervised paper summarization.\nThe first approach, based on the authors\' insight, is to use abstract knowledge, and the second approach, which is based on a knowledge-free approach.\nIn both models, given the same amount of knowledge, both approaches are able to integrate both approaches.\nFurther, we propose a method for estimating a paper\'s impact on a paper based on its authors\' knowledgeIntroductionWe describe our experience in introducing the concept of "glossy summarization" to the l

took 1m 36s when ran with cpu provided colab

##Trying with gpu


In [ ]:
def gen_pdf_summary(pdf_file_path):
    pdf_obj = PyPDF2.PdfFileReader(pdf_file_path)
    page_no = pdf_obj.getNumPages()
    text = ""
    for i in range(1, page_no):
        text += pdf_obj.getPage(i).extractText()
    
    inputs_1 = tokenizer(text[0:1024], max_length=1024, return_tensors='pt', truncation=True)
    inputs_2 = tokenizer(text[1025:2048], max_length=1024, return_tensors='pt', truncation=True)
    inputs_3 = tokenizer(text[2048:3072], max_length=1024, return_tensors='pt', truncation=True)
    inputs_4 = tokenizer(text[3072:], max_length=1024, return_tensors='pt', truncation=True)

    # Generate Summary
    summary_ids_1 = model.generate(inputs_1['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_1 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_1]

    summary_ids_2 = model.generate(inputs_2['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_2 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_2]

    summary_ids_3 = model.generate(inputs_3['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_3 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_3]

    summary_ids_4 = model.generate(inputs_4['input_ids'], num_beams=4, max_length=100, early_stopping=True)
    summary_4 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_4]
  
    summary = summary_1[0]+summary_2[0]+summary_3[0]+summary_4[0]
    return summary.strip()

In [ ]:
summary = gen_pdf_summary("/content/1909.01716.pdf")

took 1m 29s when ran with GPU

In [ ]:
print(summary)

Asolidsummaryofthe
content
From theauthors' point of
view, it may fail to understand the actual
impact
ofthe paper on
theresearch community.
Additionally, the authors of a
paper may have difficulty understanding the actual pragmatic impact
of the paper on the literature community, and may not be able to tackle the various problems in
papers summarization.
Ourcorpus, which contain the information network of ACLAnthIn this paper we present two approaches for estimating the impact of unsupervised paper summarization.
The first approach, based on the authors' insight, is to use abstract knowledge, and the second approach, which is based on a knowledge-free approach.
In both models, given the same amount of knowledge, both approaches are able to integrate both approaches.
Further, we propose a method for estimating a paper's impact on a paper based on its authors' knowledgeIntroductionWe describe our experience in introducing the concept of "glossy summarization" to the literature.
We propo